In [193]:
import pandas as pd
from pivottablejs import pivot_ui
import seaborn as sb
import matplotlib.pyplot as plt  
from ipywidgets import widgets
import numpy as np
from sklearn import (metrics, cross_validation, linear_model, preprocessing)
from sklearn.naive_bayes import BernoulliNB

%matplotlib inline

sb.set(style="ticks", color_codes=True)
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')
#g = sb.pairplot(df)

In [194]:
df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
#len(df['RESOURCE'].value_counts())
#df['RESOURCE'].plot.hist(bins=7518)

#dfc = df.iloc[:,1:10] 
dfc = df.iloc[:,:]
for col in df.columns:
  dfc[col] = df[col].astype('category')

dfc.describe()


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769,32769,32769
unique,2,7518,4243,128,177,449,343,2358,67,343
top,1,4675,770,117961,118300,117878,118321,117906,290919,118322
freq,30872,839,152,21407,4424,1135,4649,6896,10980,4649


In [4]:
#test = pd.pivot_table(dfc,index=['ROLE_TITLE','ROLE_CODE'],aggfunc='count',dropna=True)
#dfc.pivot_table(index='ROLE_TITLE',aggfunc=lambda x:len(x.unique()))
mult = {}
for c in df.columns:
    temp = df.pivot_table(index=c,aggfunc=lambda x:len(x.unique()),fill_value=0).apply(np.max)
    mult[c] = temp

multpd = pd.DataFrame(mult)


In [195]:
def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))


In [196]:

y = np.ravel(df.iloc[:,:1])
X = df.iloc[:,1:9]

X_test = dft.iloc[:,1:9].values


In [197]:
encoder = preprocessing.OneHotEncoder()
# we want to encode the category IDs encountered both in
# the training and the test set, so we fit the encoder on both
encoder.fit(np.vstack((X,X_test)))
X = encoder.transform(X)
X_test = encoder.transform(X_test)

In [198]:
print(encoder.n_values_)
print(encoder.feature_indices_)

[312154 311780 311179 286792 286793 311868 311868 308575]
[      0  312154  623934  935113 1221905 1508698 1820566 2132434 2441009]


In [191]:
SEED = 42
mean_auc = 0.0

n = 10  # repeat the CV procedure 10 times to get more precise results
#model = linear_model.LogisticRegression(C=1)
model = BernoulliNB(alpha=0.005)
for i in range(n):
    # for each iteration, randomly hold out 20% of the data as CV set
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(X, y, test_size=.20, random_state=i*SEED)

    # if you want to perform feature selection / hyperparameter
    # optimization, this is where you want to do it

    # train model and make predictions
    model.fit(X_train, y_train) 
    preds = model.predict_proba(X_cv)[:, 1]

    # compute AUC metric for this CV fold
    fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
    roc_auc = metrics.auc(fpr, tpr)
    print("AUC (fold %d/%d): %f" % (i + 1, n, roc_auc))
    mean_auc += roc_auc

print("Mean AUC: %f" % (mean_auc/n))

# === Predictions === #
# When making predictions, retrain the model on the whole training set
model.fit(X, y)
preds = model.predict_proba(X_test)[:, 1]
save_results(preds, "submit.csv")

AUC (fold 1/10): 0.840287
AUC (fold 2/10): 0.837616
AUC (fold 3/10): 0.838619
AUC (fold 4/10): 0.843097
AUC (fold 5/10): 0.823374
AUC (fold 6/10): 0.848091
AUC (fold 7/10): 0.814234
AUC (fold 8/10): 0.829486
AUC (fold 9/10): 0.820575
AUC (fold 10/10): 0.839104
Mean AUC: 0.833448


In [321]:
from itertools import combinations
data = np.vstack((df.iloc[:,1:-1],dft.iloc[:,1:-1]))
m,n = data.shape
print(m)
print(n)

yc = combinations(range(n),2)
# to view all the combinations
list(yc) 

91690
8


[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (4, 5),
 (4, 6),
 (4, 7),
 (5, 6),
 (5, 7),
 (6, 7)]

In [271]:
new_data = []
for indices in combinations(range(n),2):
    new_data.append(hash(tuple(v)) for v in data[:,indices])
    #(print(v) for v in data.iloc[:,indices])


In [228]:
list_data = []
for v in data[:,(1,2,3)]:
  list_data.append(tuple(v))
    

In [227]:
np.array(list_data)

array([[ 85475, 117961, 118300],
       [  1540, 117961, 118343],
       [ 14457, 118219, 118220],
       ..., 
       [  6114, 117961, 118343],
       [  8674, 117961, 118327],
       [  3918, 117961, 118343]], dtype=int64)

In [229]:
np.array(list_data)

array([[ 85475, 117961, 118300],
       [  1540, 117961, 118343],
       [ 14457, 118219, 118220],
       ..., 
       [  6114, 117961, 118343],
       [  8674, 117961, 118327],
       [  3918, 117961, 118343]], dtype=int64)

In [272]:
# tuple(new_data[0])

print(np.array(new_data).shape)


(56,)


In [273]:
print(np.array(new_data).T.shape)
#tuple(new_data[2])

(56,)


In [370]:
#Code to help Visualize what the algorithm is doing 
t_data = []
test_data = np.array([[1,2,3,4],[1,2,2,0],[1,2,3,4],[1,4,5,6],[2,3,4,5]])
print(test_data.shape)

#if the combination of categories are the same, they will be hashed to the same value. Eg 1 row with (0,1,0) will have same 
#hash value of another row with (0,1,0)
for indices in combinations(range(4),2):
    t_data.append(hash(tuple(v)) for v in test_data[:,indices])
    #print(v for v in test_data[:,indices])
    print(indices)
    
#Comparison 1 - Combinations are (0,1),(0,2),(0,3),(1,2),(1,3),(2,3)
# for v in test_data[:,(0,1)]:
#   print(hash(tuple(v)))

#Comparison 2 - t_data[0] corresponds to (0,1)
#tuple(t_data[0])

(5, 4)
(0, 1)
(0, 2)
(0, 3)
(1, 2)
(1, 3)
(2, 3)


In [376]:
# lst_array = np.zeros(6)
# lst_array
for col in np.array(t_data).T:
  print(col)

<generator object <genexpr> at 0x0E9EE270>
<generator object <genexpr> at 0x0E9EE240>
<generator object <genexpr> at 0x0E9EE210>
<generator object <genexpr> at 0x0E9EE3F0>
<generator object <genexpr> at 0x0E9EE180>
<generator object <genexpr> at 0x0E9EE150>


[1299869600, 1299869600, 1299869600, 1306364750, 1497441108]
[1298787075, 1299869600, 1298787075, 1305282225, 1494193533]
[1306364750, 1302034650, 1306364750, 1304199700, 1495276058]
[1497441108, 1496358583, 1497441108, 1888254024, 1699342716]
[1494193533, 1498523633, 1494193533, 1889336549, 1698260191]
[1699342716, 1498523633, 1699342716, 2081495432, 1888254024]

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [365]:
# for i,j in enumerate(t_data):
#     #lst_array[i] = np.array(list(j))
#     lst_array[i] = np.fromiter(j,np.float)

ValueError: setting an array element with a sequence.

In [378]:
# print(np.fromiter(t_data[0],np.int32))
# print(np.fromiter(t_data[1],np.int32))
# print(np.fromiter(t_data[2],np.int32))
# print(np.fromiter(t_data[3],np.int32))
# print(np.fromiter(t_data[4],np.int32))
# print(np.fromiter(t_data[5],np.int32))

ak = np.fromiter(t_data[5],np.int32)

In [380]:
aj = np.fromiter(t_data[0],np.int32)

In [382]:
set(aj)

{1299869600, 1306364750, 1497441108}

Looks like Role Title and Role Code are similar 
Use PCA ? from sklearn.decompisition import PCA


AUC = Logistic Regression returns the probability that a certain input features would be in a certain class
ROC curve visualizaes all possible thresholds
Misclassification rate is error rate for a single threshold